In [81]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)
print(latest_file)

graph_list/Town03\3_graph_v11.adjlist


In [82]:
################# Change start, end point #################

reverse = input("Do reverse search or not? (y for reverse search, other for original search)\n")

start, end = (3968, 2093), (3997, 2755)

if reverse == "y":
    start, end = end, start


In [83]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "Forward", True)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    if reverse == "y":
        fp = final_path[0][::-1]
        fp = [fp]
        add_path(G, fp, new_file_name)
    else:
        add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", rdp_img)

Start point: (3968, 2093)
End point: (3997, 2755)
Finding Path...
Not found!!!
[[(3968, 2093), (3964, 2098), (3968, 2101), (3969, 2105), (3969, 2109), (3969, 2113), (3969, 2117), (3969, 2121), (3969, 2125), (3970, 2128), (3969, 2129), (3969, 2133), (3969, 2137), (3969, 2141), (3970, 2145), (3970, 2149), (3970, 2153), (3970, 2157), (3970, 2161), (3970, 2165), (3971, 2169), (3970, 2173), (3973, 2178), (3972, 2182), (3971, 2186), (3974, 2192), (3973, 2196), (3973, 2200), (3972, 2205), (3972, 2209), (3971, 2213), (3971, 2217), (3971, 2221), (3970, 2225), (3970, 2230), (3970, 2234), (3969, 2237), (3969, 2238), (3969, 2239), (3969, 2241), (3969, 2242), (3969, 2245), (3969, 2246), (3969, 2249), (3969, 2250), (3969, 2253), (3969, 2254), (3969, 2257), (3969, 2258), (3969, 2261), (3969, 2262), (3969, 2265), (3969, 2266), (3969, 2269), (3969, 2270), (3969, 2273), (3969, 2274), (3969, 2277), (3969, 2278), (3969, 2281), (3969, 2282), (3969, 2285), (3969, 2286), (3969, 2289), (3969, 2290), (3968, 22

In [71]:
############### Quick test for other start point with the same goal point
start, end = (3664, 2052), (3997, 2755)

fp = nx.shortest_path(G,  start,  end)
print(fp)

[(3664, 2052), (3668, 2052), (3672, 2052), (3708, 2053), (3744, 2053), (3851, 2053), (3964, 2090), (3968, 2093), (3969, 2105), (3969, 2117), (3970, 2128), (3969, 2141), (4004, 2230), (4004, 2247), (4004, 2263), (4004, 2281), (4003, 2343), (4001, 2451), (4000, 2559), (3998, 2667), (3997, 2755)]
